In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv


load_dotenv(find_dotenv())
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")

In [2]:
os.environ["OPEN_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model = "llama3_instruct",
    base_url=os.getenv("OPENAI_API_BASE"),
)

### Agents

In [5]:
from tools.search_tools import *

SearchTools.search_internet

StructuredTool(name='Search the internet', description='Search the internet(query) - Useful to search the internet \n    about a a given topic and return relevant results', args_schema=<class 'pydantic.v1.main.Search the internetSchema'>, func=<function SearchTools.search_internet at 0x000001BD3B6631C0>)

In [7]:
def search_internet_duck(query):
    """Useful to search the internet 
    about a a given topic and return relevant results based on DuckDuckGo API"""
    top_result_to_return = 4
    url = "https://google.serper.dev/search"
    payload = json.dumps({"q": query})
    headers = {
        'X-API-KEY': os.environ['SERPER_API_KEY'],
        'content-type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    results = response.json()['organic']
    string = []
    for result in results[:top_result_to_return]:
      try:
        string.append('\n'.join([
            f"Title: {result['title']}", f"Link: {result['link']}",
            f"Snippet: {result['snippet']}", "\n-----------------"
        ]))
      except KeyError:
        next

    return '\n'.join(string)

In [18]:
from langchain_community.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults

In [25]:
search = DuckDuckGoSearchResults(backend= "news")

p1 = search.run("Bitcoin")

In [31]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(time="d", max_results=2)

search = DuckDuckGoSearchResults(api_wrapper=wrapper, source="news")

search.run("Bitcoin")

'[snippet: Bitcoin (BTC 1.56%) has gone through four "halvings," which halved the rewards for mining the cryptocurrency, every four years since 2012. The bulls consider those events to be major catalysts for ..., title: Is Solana a No-Brainer Buy After the Bitcoin Halving?, link: https://www.fool.com/investing/2024/07/07/is-solana-a-no-brainer-buy-after-the-bitcoin-halvi/], [snippet: The bitcoin price has dropped back from its recent all-time high of over $70,000 per bitcoin. Forbes Digital Assets. Earlier this year, Bank of America analysts warned the U.S. debt load is about ..., title: \'Worst In History\'—Bitcoin Suddenly Hurtling Toward A $1 ... - Forbes, link: https://www.forbes.com/sites/digital-assets/2024/07/07/worst-in-history-bitcoin-suddenly-hurtling-toward-a-19-trillion-shock-after-huge-price-crash/], [snippet: Bitcoin\'s price fell from $62,000 to $53,300, shocking the crypto market and killing many bullish dreams.The futures market registered long liquidations amounting t

In [35]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(time="d", max_results=2)

search = DuckDuckGoSearchResults(api_wrapper=wrapper)

search.run("Bitcoin")

RatelimitException: https://duckduckgo.com/ 202 Ratelimit

In [4]:
from crewai import Agent

from tools.browser_tools import BrowserTools
from tools.calculator_tools import CalculatorTools
from tools.search_tools import SearchTools

from langchain.tools.yahoo_finance_news import YahooFinanceNewsTool

class StockAnalysisAgents():
  def financial_analyst(self):
    return Agent(
      role='The Best Financial Analyst',
      goal="""Impress all customers with your financial data 
      and market trends analysis""",
      backstory="""The most seasoned financial analyst with 
      lots of expertise in stock market analysis and investment
      strategies that is working for a super important customer.""",
      verbose=True,
      llm = llm,
      tools=[
        SearchTools.search_internet,
        CalculatorTools.calculate,
      ]
    )

  def research_analyst(self):
    return Agent(
      role='Staff Research Analyst',
      goal="""Being the best at gather, interpret data and amaze
      your customer with it""",
      backstory="""Known as the BEST research analyst, you're
      skilled in sifting through news, company announcements, 
      and market sentiments. Now you're working on a super 
      important customer""",
      verbose=True,
      llm = llm,
      tools=[
        SearchTools.search_internet,
        SearchTools.search_news,
        YahooFinanceNewsTool(),
      ]
  )

  def investment_advisor(self):
    return Agent(
      role='Private Investment Advisor',
      goal="""Impress your customers with full analyses over stocks
      and completer investment recommendations""",
      backstory="""You're the most experienced investment advisor
      and you combine various analytical insights to formulate
      strategic investment advice. You are now working for
      a super important customer you need to impress.""",
      llm = llm,
      verbose=True,
      tools=[
        SearchTools.search_internet,
        SearchTools.search_news,
        CalculatorTools.calculate,
        YahooFinanceNewsTool()
      ]
    )

ModuleNotFoundError: No module named 'tools.sec_tools'

### Tasks

In [16]:
from crewai import Task
from textwrap import dedent

class StockAnalysisTasks():
  def research(self, agent, company):
    return Task(description=dedent(f"""
        Collect and summarize recent news articles, press
        releases, and market analyses related to the stock and
        its industry.
        Pay special attention to any significant events, market
        sentiments, and analysts' opinions. Also include upcoming 
        events like earnings and others.
  
        Your final answer MUST be a report that includes a
        comprehensive summary of the latest news, any notable
        shifts in market sentiment, and potential impacts on 
        the stock.
        Also make sure to return the stock ticker.
        
        {self.__tip_section()}
  
        Make sure to use the most recent data as possible.
  
        Selected company by the customer: {company}
      """),
      agent=agent
    )
    
  def financial_analysis(self, agent): 
    return Task(description=dedent(f"""
        Conduct a thorough analysis of the stock's financial
        health and market performance. 
        This includes examining key financial metrics such as
        P/E ratio, EPS growth, revenue trends, and 
        debt-to-equity ratio. 
        Also, analyze the stock's performance in comparison 
        to its industry peers and overall market trends.

        Your final report MUST expand on the summary provided
        but now including a clear assessment of the stock's
        financial standing, its strengths and weaknesses, 
        and how it fares against its competitors in the current
        market scenario.{self.__tip_section()}

        Make sure to use the most recent data possible.
      """),
      agent=agent
    )

  def filings_analysis(self, agent):
    return Task(description=dedent(f"""
        Analyze the latest 10-Q and 10-K filings from EDGAR for
        the stock in question. 
        Focus on key sections like Management's Discussion and
        Analysis, financial statements, insider trading activity, 
        and any disclosed risks.
        Extract relevant data and insights that could influence
        the stock's future performance.

        Your final answer must be an expanded report that now
        also highlights significant findings from these filings,
        including any red flags or positive indicators for
        your customer.
        {self.__tip_section()}        
      """),
      agent=agent
    )

  def recommend(self, agent):
    return Task(description=dedent(f"""
        Review and synthesize the analyses provided by the
        Financial Analyst and the Research Analyst.
        Combine these insights to form a comprehensive
        investment recommendation. 
        
        You MUST Consider all aspects, including financial
        health, market sentiment, and qualitative data from
        EDGAR filings.

        Make sure to include a section that shows insider 
        trading activity, and upcoming events like earnings.

        Your final answer MUST be a recommendation for your
        customer. It should be a full super detailed report, providing a 
        clear investment stance and strategy with supporting evidence.
        Make it pretty and well formatted for your customer.
        {self.__tip_section()}
      """),
      agent=agent
    )

  def __tip_section(self):
    return "If you do your BEST WORK, I'll give you a $10,000 commission!"

### MAIN

In [17]:
from crewai import Crew
from textwrap import dedent

from dotenv import load_dotenv
load_dotenv()

class FinancialCrew:
  def __init__(self, company):
    self.company = company

  def run(self):
    agents = StockAnalysisAgents()
    tasks = StockAnalysisTasks()

    research_analyst_agent = agents.research_analyst()
    financial_analyst_agent = agents.financial_analyst()
    investment_advisor_agent = agents.investment_advisor()

    research_task = tasks.research(research_analyst_agent, self.company)
    financial_task = tasks.financial_analysis(financial_analyst_agent)
    filings_task = tasks.filings_analysis(financial_analyst_agent)
    recommend_task = tasks.recommend(investment_advisor_agent)

    crew = Crew(
      agents=[
        research_analyst_agent,
        financial_analyst_agent,
        investment_advisor_agent
      ],
      tasks=[
        research_task,
        financial_task,
        filings_task,
        recommend_task
      ],
      verbose=True
    )

    result = crew.kickoff()
    return result

In [18]:
print("## Welcome to Financial Analysis Crew")
print('-------------------------------')
company = input(
  dedent("""
    What is the company you want to analyze?
  """))

financial_crew = FinancialCrew(company)
result = financial_crew.run()
print("\n\n########################")
print("## Here is the Report")
print("########################\n")
print(result)

## Welcome to Financial Analysis Crew
-------------------------------


ValidationError: 1 validation error for Task
expected_output
  Field required [type=missing, input_value={'description': "\nCollec...    important customer)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing

In [14]:
print("## Welcome to Financial Analysis Crew")
print('-------------------------------')
crypto = input(
  dedent("""
    What is the crypto you want to analyze?
  """))

financial_crew = FinancialCrew(crypto)
result = financial_crew.run()
print("\n\n########################")
print("## Here is the Report")
print("########################\n")
print(result)

## Welcome to Financial Analysis Crew
-------------------------------


ValidationError: 1 validation error for Task
expected_output
  Field required [type=missing, input_value={'description': "\nCollec...ortant customer
      )}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing

In [ ]:
import o

In [2]:
"""
CryptoPanic API Wrapper.

Docs for API found here. https://cryptopanic.com/developers/api/

CryptoPanic Server is cached every 30 seconds. So crawling at a faster rate is pointless.
All API methods are rate limited per IP at 5req/sec.

Use like this:

    df = concatenate_pages(pages_list=get_pages_list_json(2, make_url(filter='hot', currencies='btc,eth,xrp')))

"""

import requests
import pandas as pd
import datetime
import time

global_api_rate_delay = .2  # All API methods are rate limited per IP at 5req/sec.


def make_url(filter=None, currencies=None, kind=None, region=None, page=None):
    """Handle of URL variables for API POST."""
    url = f"""https://cryptopanic.com/api/v1/posts/?auth_token={os.getenv("CRYPTOPANIC_API_KEY")}"""

    if currencies is not None:
        if len(currencies.split(',')) <= 50:
            url += "&currencies={}".format(currencies)
        else:
            print("Warning: Max Currencies is 50")
            return

    if kind is not None and kind in ['news', 'media']:
        url += "&kind={}".format(kind)

    filters = ['rising', 'hot', 'bullish', 'bearish', 'important', 'saved', 'lol']
    if filter is not None and filter in filters:
        url += "&filter={}".format(filter)

    regions = ['en', 'de', 'es', 'fr', 'it', 'pt', 'ru']  # (English), (Deutsch), (Español), (Français), (Italiano), (Português), (Русский)--> Respectively
    if region is not None and region in regions:
        url += "&region={}".format(region)

    if page is not None:
        url += "&page={}".format(page)

    return url


def get_page_json(url=None):
    """
    Get First Page.

    Returns Json.

    """
    time.sleep(global_api_rate_delay)
    if not url:
        url =f"""https://cryptopanic.com/api/v1/posts/?auth_token={os.getenv("CRYPTOPANIC_API_KEY")}"""
    page = requests.get(url)
    data = page.json()
    return data


def get_pages_list_json(lookback, url):
    """
    Get history of pages starting from page 1 to the lookback.

    Returns: List of Pages in Json format

    """
    pages_list_json = [get_page_json(url)]

    for i in range(lookback):
        pages_list_json.append(get_page_json(pages_list_json[i]["next"]))

    return pages_list_json


def get_df(data):
    """Return pandas DF."""
    df = pd.DataFrame(data)
    try:
        df['created_at'] = pd.to_datetime(df.created_at)
    except Exception as e:
        pass

    return df


def concatenate_pages(pages_list):
    """Concatenate Pages into one Dataframe."""
    frames = []
    for page in pages_list:
        frames.append(get_df(page))

    return pd.concat(frames, ignore_index=True)

# df = concatenate_pages(get_pages_list_json(2, make_url()))

In [13]:
df = concatenate_pages(get_pages_list_json(2, make_url(currencies = 'BTC', filter= 'hot')))

In [16]:
df.iloc[0]['results']

{'kind': 'news',
 'domain': 'cryptobriefing.com',
 'votes': {'negative': 2,
  'positive': 3,
  'important': 2,
  'liked': 4,
  'disliked': 0,
  'lol': 2,
  'toxic': 0,
  'saved': 0,
  'comments': 4},
 'source': {'title': 'CryptoBriefing',
  'region': 'en',
  'domain': 'cryptobriefing.com',
  'path': None},
 'title': 'Mt. Gox confirms Bitcoin, Bitcoin Cash repayments have begun',
 'published_at': '2024-07-05T06:48:03Z',
 'slug': 'Mt-Gox-confirms-Bitcoin-Bitcoin-Cash-repayments-have-begun',
 'currencies': [{'code': 'BTC',
   'title': 'Bitcoin',
   'slug': 'bitcoin',
   'url': 'https://cryptopanic.com/news/bitcoin/'},
  {'code': 'BCH',
   'title': 'Bitcoin Cash',
   'slug': 'bitcoin-cash',
   'url': 'https://cryptopanic.com/news/bitcoin-cash/'}],
 'id': 19656255,
 'url': 'https://cryptopanic.com/news/19656255/Mt-Gox-confirms-Bitcoin-Bitcoin-Cash-repayments-have-begun',
 'created_at': '2024-07-05T06:48:03Z',
 'metadata': {'description': 'Read more about this article directly on the news pa

In [17]:
url_everything = make_url(currencies = 'BTC', filter= 'hot')
everything_pages_last_100 = get_pages_list_json(9, url_everything)

everything_pages_last_100_results = []
for page in everything_pages_last_100:
    everything_pages_last_100_results.append(page['results'])
df_results = concatenate_pages(everything_pages_last_100_results)

In [18]:
df_results

,kind,domain,votes,source,title,published_at,slug,currencies,id,url,created_at,metadata
0,news,cryptobriefing.com,"{'negative': 2, 'positive': 3, 'important': 2,...","{'title': 'CryptoBriefing', 'region': 'en', 'd...","Mt. Gox confirms Bitcoin, Bitcoin Cash repayme...",2024-07-05T06:48:03Z,Mt-Gox-confirms-Bitcoin-Bitcoin-Cash-repayment...,"[{'code': 'BTC', 'title': 'Bitcoin', 'slug': '...",19656255,https://cryptopanic.com/news/19656255/Mt-Gox-c...,2024-07-05 06:48:03+00:00,{'description': 'Read more about this article ...
1,news,coinedition.com,"{'negative': 6, 'positive': 8, 'important': 5,...","{'title': 'coinedition', 'region': 'en', 'doma...",Cardano (ADA) vs. Bitcoin (BTC): Which Crypto ...,2024-07-04T10:40:00Z,Cardano-ADA-vs-Bitcoin-BTC-Which-Crypto-is-a-B...,"[{'code': 'BTC', 'title': 'Bitcoin', 'slug': '...",19653504,https://cryptopanic.com/news/19653504/Cardano-...,2024-07-04 10:40:00+00:00,{'description': 'Read more about this article ...
2,news,bitcoinist.com,"{'negative': 5, 'positive': 22, 'important': 1...","{'title': 'Bitcoinist', 'region': 'en', 'domai...",BREAKING: Donald Trump Touts Bitcoin As Strate...,2024-07-03T20:13:22Z,BREAKING-Donald-Trump-Touts-Bitcoin-As-Strateg...,"[{'code': 'BTC', 'title': 'Bitcoin', 'slug': '...",19651447,https://cryptopanic.com/news/19651447/BREAKING...,2024-07-03 20:13:22+00:00,{'description': 'Read more about this article ...
3,news,coinpaper.com,"{'negative': 1, 'positive': 6, 'important': 1,...","{'title': 'Coinpaper', 'region': 'en', 'domain...",Ethereum Predicted to Outperform Bitcoin After...,2024-07-03T07:30:00Z,Ethereum-Predicted-to-Outperform-Bitcoin-After...,"[{'code': 'BTC', 'title': 'Bitcoin', 'slug': '...",19648586,https://cryptopanic.com/news/19648586/Ethereum...,2024-07-03 07:30:00+00:00,{'description': 'Read more about this article ...
4,news,newsbtc.com,"{'negative': 4, 'positive': 4, 'important': 1,...","{'title': 'NewsBTC', 'region': 'en', 'domain':...",Cardano (ADA) On The Rise: Exploring Upward Mo...,2024-07-03T04:58:12Z,Cardano-ADA-On-The-Rise-Exploring-Upward-Momen...,"[{'code': 'BTC', 'title': 'Bitcoin', 'slug': '...",19648190,https://cryptopanic.com/news/19648190/Cardano-...,2024-07-03 04:58:12+00:00,{'description': 'Read more about this article ...
...,...,...,...,...,...,...,...,...,...,...,...,...
195,news,zycrypto.com,"{'negative': 0, 'positive': 9, 'important': 3,...","{'title': 'ZyCrypto', 'region': 'en', 'domain'...","Binance CEO Predicts Bitcoin Rally Past $80,00...",2024-03-17T14:11:07Z,Binance-CEO-Predicts-Bitcoin-Rally-Past-80000-...,"[{'code': 'BTC', 'title': 'Bitcoin', 'slug': '...",19334030,https://cryptopanic.com/news/19334030/Binance-...,2024-03-17 14:11:07+00:00,{'description': 'Read more about this article ...
196,news,dailycoin.com,"{'negative': 0, 'positive': 4, 'important': 6,...","{'title': 'Dailycoin', 'region': 'en', 'domain...",El Salvador's Bitcoin Vault Reveals Way More B...,2024-03-15T22:06:00Z,El-Salvadors-Bitcoin-Vault-Reveals-Way-More-BT...,"[{'code': 'BTC', 'title': 'Bitcoin', 'slug': '...",19331822,https://cryptopanic.com/news/19331822/El-Salva...,2024-03-15 22:06:00+00:00,{'description': 'Read more about this article ...
197,news,newsbtc.com,"{'negative': 0, 'positive': 9, 'important': 4,...","{'title': 'NewsBTC', 'region': 'en', 'domain':...","Bitcoin Suffers Crash To $65,000 As Spot ETF I...",2024-03-15T14:30:56Z,Bitcoin-Suffers-Crash-To-65000-As-Spot-ETF-Inf...,"[{'code': 'BTC', 'title': 'Bitcoin', 'slug': '...",19330810,https://cryptopanic.com/news/19330810/Bitcoin-...,2024-03-15 14:30:56+00:00,{'description': 'Read more about this article ...
198,news,protos.com,"{'negative': 0, 'positive': 6, 'important': 5,...","{'title': 'Protos.com', 'region': 'en', 'domai...","SEC website taken offline, precedes bitcoin crash",2024-03-15T11:15:14Z,SEC-website-taken-offline-precedes-bitcoin-crash,"[{'code': 'BTC', 'title': 'Bitcoin', 'slug': '...",19330155,https://cryptopanic.com/news/19330155/SEC-webs...,2024-03-15 11:15:14+00:0

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

